In [1]:
#Conectar Python a MySQL (Sakila)
import pandas as pd
from sqlalchemy import create_engine

# Reemplaza con tus credenciales de conexión
user = 'root'
password = 'isabelyluis'
host = 'localhost'
port = 3306
database = 'sakila'

# Crear motor de conexión
engine = create_engine(f'mysql+pymysql://{user}:{"isabelyluis"}@{host}:{port}/{"sakila"}')


In [ ]:
#funcion rentals_month
def rentals_month(engine, month, year):
    query = f"""
    SELECT rental_id, rental_date, customer_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df = pd.read_sql(query, con=engine)
    return df

In [3]:
#funcion rental count month 
def rental_count_month(df, month, year):
    col_name = f"rentals_{str(month).zfill(2)}_{year}"
    count_df = df.groupby('customer_id').size().reset_index(name=col_name)
    return count_df


In [4]:
#funcion comparacion de rentas por mes y año 
def compare_rentals(df1, df2):
    combined = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    # Suponemos que las columnas de los DataFrames son del estilo rentals_05_2005
    col1 = combined.columns[1]
    col2 = combined.columns[2]
    combined['difference'] = combined[col2] - combined[col1]
    return combined



In [5]:
# Obtener datos de mayo y junio 2005
df_may = rentals_month(engine, 5, 2005)
df_june = rentals_month(engine, 6, 2005)

# Contar alquileres por cliente
count_may = rental_count_month(df_may, 5, 2005)
count_june = rental_count_month(df_june, 6, 2005)

# Comparar ambos meses
result = compare_rentals(count_may, count_june)
print(result.head())


   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1              2.0              7.0         5.0
1            2              1.0              1.0         0.0
2            3              2.0              4.0         2.0
3            4              0.0              6.0         6.0
4            5              3.0              5.0         2.0
